# Data Cleaning

In [1]:
import pandas as pd 

In [2]:
df = pd.read_csv('../csvs/season/all_team_standings.csv')
df

,Season,Team ID,Team Name,Owner(s),Wins,Losses,Points For,Points Against,Regular Season Rank,Final Rank
0,2012,1,Percy Whipped,Samuel Remler,7,6,1433.74,1363.62,3,1
1,2012,4,Forgetting BrandonMarshall,Jesse Hynes,7,6,1347.80,1281.94,4,2
2,2012,2,The Dawgs,"Julian Bombard, julian bombard",8,5,1263.98,1161.24,2,3
3,2012,7,Joe Buck Yourself,Hunter Rieger,8,5,1308.00,1189.54,1,4
4,2012,6,The Madd Prater,"kellen dreyer, Parker Denison",6,7,1234.48,1294.80,8,5
...,...,...,...,...,...,...,...,...,...,...
147,2024,3,Fergayson or th0t daughter,"daniel kvassov, Jake Price",7,7,1410.14,1395.26,7,8
148,2024,10,Rectum Wrecker$$$,"K S, Alex Rowland",4,10,1068.44,1342.72,12,9
149,2024,7,Kamara Harris,"Isaac Rothenberg, Keyan Rahim",5,9,1320.60,1338.20,10,10
150,2024,4,ham in a tube,Jesse Hynes,5,9,1276.60,1431.32,11,11


### Step 1: Normalize Team Owner Names

Map correct owner names for each season

In [3]:
import pandas as pd

def expand_owner_ranges(rows):
    out = []
    for r in rows:
        for yr in range(r["Start"], r["End"] + 1):
            out.append({"Season": yr, "Team ID": r["Team ID"], "Owner(s)": r["Owner(s)"]})
    return pd.DataFrame(out).sort_values(["Team ID", "Season"]).reset_index(drop=True)

owner_ranges = [
    # Team 1
    {"Team ID": 1,  "Start": 2012, "End": 2024, "Owner(s)": "Samuel Remler"},
    # Team 2
    {"Team ID": 2,  "Start": 2012, "End": 2024, "Owner(s)": "Julian Bombard"},
    # Team 3
    {"Team ID": 3,  "Start": 2012, "End": 2012, "Owner(s)": "Daniel Kvassov"},
    {"Team ID": 3,  "Start": 2013, "End": 2017, "Owner(s)": "Daniel Kvassov, Aidan Donahue"},
    {"Team ID": 3,  "Start": 2018, "End": 2024, "Owner(s)": "Daniel Kvassov, Jake Price"},
    # Team 4
    {"Team ID": 4,  "Start": 2012, "End": 2024, "Owner(s)": "Jesse Hynes"},
    # Team 5
    {"Team ID": 5,  "Start": 2012, "End": 2012, "Owner(s)": "Aidan Donahue"},
    {"Team ID": 5,  "Start": 2013, "End": 2024, "Owner(s)": "Yassine Hamdouni"},
    # Team 6
    {"Team ID": 6,  "Start": 2012, "End": 2024, "Owner(s)": "Kellen Dreyer, Tal Litwin"},
    # Team 7
    {"Team ID": 7,  "Start": 2012, "End": 2014, "Owner(s)": "Hunter Rieger"},
    {"Team ID": 7,  "Start": 2015, "End": 2017, "Owner(s)": "Hunter Rieger, Jake Price"},
    {"Team ID": 7,  "Start": 2018, "End": 2023, "Owner(s)": "Keyan Rahim"},
    {"Team ID": 7,  "Start": 2024, "End": 2024, "Owner(s)": "Keyan Rahim, Isaac Rothenberg"},
    # Team 8
    {"Team ID": 8,  "Start": 2012, "End": 2024, "Owner(s)": "Jacob Maler, Nathan Zicherman"},
    # Team 9
    {"Team ID": 9,  "Start": 2012, "End": 2024, "Owner(s)": "Lorenzo Siemann"},
    # Team 10
    {"Team ID": 10, "Start": 2012, "End": 2024, "Owner(s)": "Alex Rowland"},
    # Team 11
    {"Team ID": 11, "Start": 2012, "End": 2024, "Owner(s)": "Reuben Goldberg"},
    # Team 12
    {"Team ID": 12, "Start": 2014, "End": 2017, "Owner(s)": "Liam Pauley"},
    {"Team ID": 12, "Start": 2018, "End": 2024, "Owner(s)": "Jasper Hebert, Joe Kahn"},
]

owners_map = expand_owner_ranges(owner_ranges)
owners_map.head(), owners_map.tail()


(   Season  Team ID       Owner(s)
 0    2012        1  Samuel Remler
 1    2013        1  Samuel Remler
 2    2014        1  Samuel Remler
 3    2015        1  Samuel Remler
 4    2016        1  Samuel Remler,
      Season  Team ID                 Owner(s)
 149    2020       12  Jasper Hebert, Joe Kahn
 150    2021       12  Jasper Hebert, Joe Kahn
 151    2022       12  Jasper Hebert, Joe Kahn
 152    2023       12  Jasper Hebert, Joe Kahn
 153    2024       12  Jasper Hebert, Joe Kahn)

Apply mapped owners to standings and save cleaned file 

In [4]:
standings_clean = df.merge(
    owners_map,
    on=["Season", "Team ID"],
    how="left",
    suffixes=("", "_mapped")
)

standings_clean["Owner(s)_Clean"] = standings_clean["Owner(s)_mapped"].fillna(standings_clean["Owner(s)"])

cols_order = [
    "Season","Team ID","Team Name","Owner(s)_Clean","Wins","Losses",
    "Points For","Points Against","Regular Season Rank","Final Rank"
]
standings_clean = standings_clean.reindex(columns=cols_order)

standings_clean.to_csv("../csvs/season/all_team_standings_cleaned.csv", index=False)
print("✅ Saved: ../csvs/season/all_team_standings_cleaned.csv")


✅ Saved: ../csvs/season/all_team_standings_cleaned.csv


### Step 2: Fix Playoff Matchup Columns 

Setup and load data

In [16]:
import pandas as pd
import glob
import os

standings = pd.read_csv("../csvs/season/all_team_standings_cleaned.csv")

matchup_files = sorted(glob.glob("../csvs/matchups/matchups_*.csv"))
matchups = pd.concat((pd.read_csv(f) for f in matchup_files), ignore_index=True)

matchups["Season"] = matchups["Season"].astype(int)
matchups["Week"] = matchups["Week"].astype(int)

for col in ["Is Bye Week", "Is Playoff", "Is Consolation"]:
    if col not in matchups.columns:
        matchups[col] = False


Determine which matchups are playoff and consolation

In [17]:
def playoff_seeds_for_season(season: int) -> int:
    # 2012–2013: 4 seeds; 2014+: 6 seeds
    return 4 if season in (2012, 2013) else 6

def postseason_weeks_for_season(season: int) -> range:
    # Pre-2021: weeks 14–16; 2021+: weeks 15–17
    if season >= 2021:
        return range(15, 18)  # 15,16,17
    else:
        return range(14, 17)  # 14,15,16

def has_first_round_byes(season: int) -> bool:
    # Only in 6-seed formats (i.e., 2014+)
    return season >= 2014


Compute playoff and consolation

In [18]:
seed_info = standings[["Season", "Team ID", "Regular Season Rank"]]
mx = matchups.merge(seed_info, on=["Season","Team ID"], how="left")

mx["Is_Postseason"] = False
mx["Is_PlayoffTeam"] = False
mx["Is Playoff (Computed)"] = False
mx["Is Consolation (Computed)"] = False

out_frames = []

for season, df_y in mx.groupby("Season", sort=True):
    df_y = df_y.sort_values("Week").copy()
    seeds = playoff_seeds_for_season(season)
    ps_weeks = set(postseason_weeks_for_season(season))
    first_round_byes = has_first_round_byes(season)

    df_y["Is_Postseason"] = df_y["Week"].isin(ps_weeks)

    df_y["Is_PlayoffTeam"] = df_y["Regular Season Rank"] <= seeds

    team_status = {
        tid: ("playoff" if (rank <= seeds) else "consolation")
        for tid, rank in df_y.drop_duplicates("Team ID").set_index("Team ID")["Regular Season Rank"].items()
    }

    top2 = set(
        df_y[df_y["Regular Season Rank"].isin([1,2])]["Team ID"].unique()
    ) if (seeds == 6 and first_round_byes) else set()

    for wk, df_w in df_y.groupby("Week", sort=True):
        postseason_week = wk in ps_weeks

        to_consolation = set()

        for idx, row in df_w.iterrows():
            tid = row["Team ID"]
            opp = row.get("Opponent ID", None)
            score = row.get("Score", None)
            opp_score = row.get("Opponent Score", None)
            is_bye = bool(row.get("Is Bye Week", False))
            status = team_status.get(tid, "consolation")

            if not postseason_week:
                continue

            if status == "playoff":
                df_y.loc[idx, "Is Playoff (Computed)"] = True

                if is_bye and tid in top2:
                    continue

                if pd.notna(score) and pd.notna(opp_score):
                    try:
                        if float(score) < float(opp_score):
                            to_consolation.add(tid)
                    except Exception:
                        pass

            else:
                df_y.loc[idx, "Is Consolation (Computed)"] = True

        for loser_id in to_consolation:
            team_status[loser_id] = "consolation"

    mask_post = df_y["Is_Postseason"]
    df_y.loc[mask_post & ~df_y["Is Playoff (Computed)"] & ~df_y["Is_PlayoffTeam"], "Is Consolation (Computed)"] = True

    out_frames.append(df_y)

matchups_clean = pd.concat(out_frames, ignore_index=True)


Enrich matchup data with owner names and organize columns

In [19]:
# Add owner names if you like
owners = standings[["Season", "Team ID", "Owner(s)_Clean"]].drop_duplicates()
matchups_clean = matchups_clean.merge(owners, on=["Season","Team ID"], how="left")

# Choose a neat column order
keep_cols = [
    "Season","Week",
    "Team ID","Team Name","Owner(s)_Clean",
    "Opponent ID","Opponent Name",
    "Score","Opponent Score",
    "Is Home","Is Bye Week",
    # flags from scraper (if present)
    "Is Playoff","Is Consolation",
    # computed flags
    "Is_Postseason","Is_PlayoffTeam","Is Playoff (Computed)","Is Consolation (Computed)"
]
matchups_clean = matchups_clean[[c for c in keep_cols if c in matchups_clean.columns]]


Drop defunct columns and save in single combined CSV

In [26]:
matchups_final = matchups_clean.rename(
    columns={
        "Is Playoff (Computed)": "IsPlayoff",
        "Is Consolation (Computed)": "IsConsolation",
    }
)

drop_cols = ["Is Playoff", "Is Consolation", "Is_Postseason", "Is_PlayoffTeam"]
matchups_final = matchups_final.drop(columns=[c for c in drop_cols if c in matchups_final.columns])

final_cols = [
    "Season","Week",
    "Team ID","Team Name","Owner(s)_Clean",
    "Opponent ID","Opponent Name",
    "Score","Opponent Score",
    "Is Home","Is Bye Week",
    "IsPlayoff","IsConsolation"
]
matchups_final = matchups_final[[c for c in final_cols if c in matchups_final.columns]].copy()

out_path = "../csvs/matchups/matchups_all_cleaned.csv"
matchups_final.to_csv(out_path, index=False)
print(f"✅ Saved clean matchups CSV: {out_path}  (rows={len(matchups_final):,})")



✅ Saved clean matchups CSV: ../csvs/matchups/matchups_all_cleaned.csv  (rows=2,460)


In [27]:
df2 = pd.read_csv('../csvs/matchups/matchups_all_cleaned.csv')
df2

,Season,Week,Team ID,Team Name,Owner(s)_Clean,Opponent ID,Opponent Name,Score,Opponent Score,Is Home,Is Bye Week,IsPlayoff,IsConsolation
0,2012,1,1,Percy Whipped,Samuel Remler,6.0,The Madd Prater,111.90,93.88,True,False,False,False
1,2012,1,6,The Madd Prater,"Kellen Dreyer, Tal Litwin",1.0,Percy Whipped,93.88,111.90,False,False,False,False
2,2012,1,5,Mister Rodgers Neighborhood,Aidan Donahue,2.0,The Dawgs,77.32,81.60,True,False,False,False
3,2012,1,2,The Dawgs,Julian Bombard,5.0,Mister Rodgers Neighborhood,81.60,77.32,False,False,False,False
4,2012,1,10,What would Jones Drew?,Alex Rowland,7.0,Joe Buck Yourself,92.60,98.78,True,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2455,2024,17,6,Andrew Luck Memorial Squad,"Kellen Dreyer, Tal Litwin",9.0,Purdy Ladds,160.58,121.54,False,False,True,False
2456,2024,17,9,Purdy Ladds,Lorenzo Siemann,6.0,Andrew Luck Memorial Squad,121.54,160.58,True,False,True,False
2457,2024,17,12,Its literally too easy,"Jasper Hebert, Joe Kahn",4.0,ham in a tube,71.68,99.78,True,False,False,True
2458,2024,17,8,The Healthy Colons,"Jacob Maler, Nathan Zicherman",5.0,brb building rome,120.18,81.86,True,False,False,True


In [28]:
df2['IsPlayoff'].value_counts()

IsPlayoff
False    2316
True      144
Name: count, dtype: int64